In [ ]:
import pandas as pd  
import os                
from datetime import datetime
import time
import matplotlib.pyplot as plt
from math import sin, cos, tan
import matplotlib.dates as mdates

#### check the data structure of all the files 

In [ ]:

# loop through directory where your files are located
directory = './Data/'
dfs = []
i = 1
#loop through all the files in a data directory
for filename in os.listdir(directory):
    #ignore a file start with the name weekly, thats the mortageg rates data 
    if not filename.startswith("weekly"):
        #read file path 
        filepath = os.path.join(directory, filename)
        #read a file in a dataframe
        df = pd.read_csv(filepath)
        #append the file in dictionary 
        print(f"DataFrame {i} structure:")
        df.info()
        print("\n---\n")
        i =+ i
              



##### Column structure is consistent across all DFs. 
##### Dataframe 2 (2021q2) contains the largest set of data at 14862 entries.
##### Dataframe 3 (2021q3) comes in at close second at 14693 entries. 
##### 18 columns for each Df. 


In [ ]:
# Directory where your files are located
directory = './Data/'
dfs = []

#loop through all the files in a data directory
for filename in os.listdir(directory):
    #ignore a file start with the name weekly, thats the mortageg rates data 
    if not filename.startswith("weekly"):
        #read file path 
        filepath = os.path.join(directory, filename)
        #read a file in a dataframe
        df = pd.read_csv(filepath)
        #append the file in dictionary 
        dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)    
combined_df.head()

In [ ]:
#Checking how many values are null per column
combined_df.isnull().sum()

In [ ]:
#Checking basic structure of df
combined_df.shape

##### 162138 rows
##### 18 columns

In [ ]:
#Checking data types of the columns
combined_df.dtypes

##### Zipcode should stay as object due to certain areas formatting for zip codes such as 27006-8256
##### Total living area sqft is currently an object--Should be a float?
##### Should convert year built, list date, closing date to date time format rather than float/objects. 
##### List price and sold price columns should be converted to floats and have "$" stripped for graphing/numerical purposes.
##### Acres should be converted to float and have "Acres" stripped for same reason as list price. 

In [ ]:

# Due the existence of 17 null values in the "Year Built" column we should convert these values to floats with a value of "0"
#   prior to the conversion to datetime to avoid any errors. 
combined_df['Year Built'] = combined_df['Year Built'].fillna(0).astype(float)
#Here I convert the recently converted columns that now contain "0" to "0000" for a more seamless datetime conversion.
#   For now we will assume empty columns contain the value "0000". This must be reconsidered for any impact it might have on future analytics. 
combined_df['Year Built'] = combined_df['Year Built'].replace('0', '0000')

# converting "Year Built" from float to datetime. For now I am assuming jan 1 for the day/month
combined_df['Year Built'] = pd.to_datetime(combined_df['Year Built'], format='%Y', errors='coerce')

# converting list date and closing date from str to datetime format. 
combined_df['List Date'] = pd.to_datetime(combined_df['List Date'], format='%m/%d/%Y')
combined_df['Closing Date'] = pd.to_datetime(combined_df['Closing Date'], format='%m/%d/%Y')

In [ ]:
combined_df.dtypes

In [ ]:
combined_df

##### I am receiving a key error on List Price and Sold Price and have tried copy pasting among other methods so my assumption is that there might be hidden characters or leading spaces.
#####       For this reason I will be renaming the columns manually to ensure they are contingent with our eyes. 

In [ ]:
#Obtaining index values of columns. 
combined_df.info()

In [ ]:
#confirming that index values are correct
print(f"List price should equal {combined_df.columns[16]} and Sold price should equal {combined_df.columns[17]}")

In [ ]:
combined_df.rename(columns={combined_df.columns[16]: "List Price"}, inplace=True)
combined_df.rename(columns={combined_df.columns[17]: "Sold Price"}, inplace=True)
combined_df


In [ ]:
combined_df["List Price"] = combined_df["List Price"].str.replace("$", "").str.replace(',', '').astype(float)
combined_df["Sold Price"] = combined_df["Sold Price"].str.replace("$", "").str.replace(',', '').astype(float)

combined_df

##### List price and sold price have successfully been renamed and converted to floats.
#####
##### Now I will do the same conversion for acres. 
##### Since acres is a range value it is not possible to display this in a single column as floats are single numeric values. 
##### For this reason I will be splitting the acres column into two separate columns, one for min and one for max.  
##### We can average these two columns for graphing purposes. 

In [ ]:
combined_df.info()

In [ ]:
combined_df.rename(columns={combined_df.columns[11]: "Acres"}, inplace=True)
#Key value error prevention. It seems some column names contain trailing spaces or hidden characters so i'm just going to do this ahead of time. 

In [ ]:
def split_range(range_str):
    if isinstance(range_str, str):
        range_str = range_str.strip()  # removes leading/trailing whitespace
        if range_str.endswith('+'):
            range_str = range_str[:-1].strip()  # removes trailing '+' and any whitespace
        parts = range_str.split('-')
        if len(parts) == 1:
            return [float(parts[0]), float('inf')]  # use infinity for upper bound on values such as "11+" which is an error I received when creating this function. I think Python assumes 11+ means 11-infinity.
        else:
            return [float(parts[0]), float(parts[1])]
    else:
        return [float('nan'), float('nan')]
combined_df[['Acres min', 'Acres max']] = combined_df['Acres'].str.replace(' Acres', '').apply(split_range).tolist()

In [ ]:
combined_df

In [ ]:
combined_df.info()

In [ ]:
combined_df["Total Living Area SqFt"] = combined_df["Total Living Area SqFt"].str.replace(",", "").astype(float)

### Creating "Season" columns and grouping into the new season columns

In [ ]:
def get_season(date):
    if date.month in [3, 4, 5]:
        return 'Spring'
    elif date.month in [6, 7, 8]:
        return 'Summer'
    elif date.month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

combined_df['Season Listed'] = combined_df['List Date'].apply(get_season)
combined_df['Season Sold'] = combined_df['Closing Date'].apply(get_season)
combined_df

### Graphing List price over time

In [ ]:
#here I am filtering the data frame so that the graph only displays 2021-now. I had some issues while graphing where the graph was going all the way back to 1978 so it is necessary to filter
filtered_df = combined_df[(combined_df['List Date'] >= pd.Timestamp('2021-01-01')) & (combined_df['List Date'] <= pd.Timestamp.now())]

#here i am grouping the data by months so that it is easier for us to read. When I plotted it without grouping it resulted in a graph that looks like I had given a monkey a pencil and told it to draw the mona lisa.
monthly_data_list = filtered_df.resample('M', on='List Date').agg({'List Price': ['sum', 'count']})
monthly_data_sold = filtered_df.resample('M', on='List Date').agg({'Sold Price': ['sum', 'count']})
monthly_data_list.columns = ['Total List Price', 'Total Properties Listed']
monthly_data_sold.columns = ['Total Sale Price', 'Total Properties Listed']


In [ ]:
#setting the fig size. had to play around a bit to find a good fit.
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot that displays the sum of total sales price by month.
color = 'tab:blue'
ax1.set_xlabel('Month')
ax1.set_ylabel('Sum of Total Sale Price', color=color)
ax1.plot(monthly_data_sold.index, monthly_data_sold['Total Sale Price'], color=color, marker='x')
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_title('Sum of Total List & Sales Prices Per Month from 2021 to Present')



# twin axes for the sum of the list prices
# telling the program that the axes will be plotted on the same graph. 
ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Sum of Total List Price', color=color)
ax2.plot(monthly_data_list.index, monthly_data_list['Total List Price'], color=color, marker='o')
ax2.tick_params(axis='y', labelcolor=color)


#tightening the layout
fig.tight_layout()  
print(monthly_data_list)
plt.show()

In [ ]:
# bar chart that displays total properties listed per month.
fig, sales_volume = plt.subplots(figsize=(12, 6))
color = 'tab:blue'
sales_volume.set_xlabel('Month')
sales_volume.set_ylabel('Total Properties Listed', color=color)
sales_volume.bar(monthly_data_list.index, monthly_data_list['Total Properties Listed'], color=color)
sales_volume.tick_params(axis='y', labelcolor=color)
sales_volume.set_title('Total Properties Listed by Month')


### Fixing winston-salem vs winston salem rows:

In [ ]:
combined_df = combined_df.replace(["Winston Salem"], "Winston-Salem")

In [ ]:
combined_df.loc[combined_df["City"] == "Winston-Salem"]

In [ ]:
city_counts = combined_df['City'].value_counts()

#resetting index to check alphabetically
city_counts_df = city_counts.reset_index()
#making new column to show occurrences
city_counts_df.columns = ['City', 'Occurrences']

#sorting alphabetically
city_counts_df = city_counts_df.sort_values('City')


city_counts_df.to_csv('City_Occurrences.csv', index=False)

###### Winston-Salem looks to be the only duplicate city name.

#### Grouping by City, Zip, Sold Price:
#### Grouping by City, Zip, Days on market:

In [ ]:
sold_price_grouped = combined_df[["City", "Zip", "Sold Price"]].sort_values(by="Sold Price", ascending=True)
DaysOnMarket_grouped = combined_df[["City", "Zip", "Days On Market"]].sort_values(by="Days On Market", ascending=True)

### Days on market grouped for all cities/zipcodes:
### Sold Price grouped for all cities/zipcodes:

In [ ]:
display(DaysOnMarket_grouped, sold_price_grouped)

####  DOM = days on market
#### Sold Price = SP

In [ ]:
raleigh_grouped_all = combined_df.loc[combined_df["City"] == "Raleigh"]
raleigh_grouped_SP = sold_price_grouped.loc[sold_price_grouped["City"] == "Raleigh"]
raleigh_grouped_DOM = DaysOnMarket_grouped.loc[DaysOnMarket_grouped["City"] == "Raleigh"]
raleigh_grouped_all

fayetteville_grouped_all = combined_df.loc[combined_df["City"] == "Fayetteville"]
fayetteville_grouped_SP = sold_price_grouped.loc[sold_price_grouped["City"] == "Fayetteville"]
fayetteville_grouped_DOM = DaysOnMarket_grouped.loc[DaysOnMarket_grouped["City"] == "Fayetteville"]
fayetteville_grouped_all

### Raleigh Sold price:
### Raleigh Days on market:

In [ ]:
display(raleigh_grouped_SP, raleigh_grouped_DOM)

In [ ]:
display(fayetteville_grouped_SP, raleigh_grouped_DOM)

### Creating filtered DataFrames for Raleigh & Fayetteville; removing unnecessary columns:

In [ ]:
raleigh_grouped_3_2 = raleigh_grouped_all.loc[
    (raleigh_grouped_all["Bedrooms"] == 3.0) &
    (raleigh_grouped_all["Total Baths"] == 2.0)
]
raleigh_grouped_3_2 = raleigh_grouped_3_2[[
    "MLS #", "Property Type", "City", "Bedrooms", 
    "Total Baths", "Total Living Area SqFt",
    "Year Built", "List Date", "Closing Date",
    "Days On Market", "List Price", "Sold Price", "Acres min", "Acres max",
    "Season Listed", "Season Sold"
    ]]

to_drop = raleigh_grouped_3_2[raleigh_grouped_3_2["Property Type"] == "Duplex"].index
raleigh_grouped_3_2 = raleigh_grouped_3_2.drop(to_drop)
raleigh_grouped_3_2


single_family_residence_3_2 = raleigh_grouped_3_2.loc[raleigh_grouped_3_2['Property Type'] == 'Single Family Residence']
single_family_residence_3_2

##################################################################

fayetteville_grouped_3_2 = fayetteville_grouped_all.loc[
    (fayetteville_grouped_all["Bedrooms"] == 3.0) &
    (fayetteville_grouped_all["Total Baths"] == 2.0)
]
fayetteville_grouped_3_2 = fayetteville_grouped_3_2[[
    "MLS #", "Property Type", "City", "Bedrooms", 
    "Total Baths", "Total Living Area SqFt",
    "Year Built", "List Date", "Closing Date",
    "Days On Market", "List Price", "Sold Price", "Acres min", "Acres max",
    "Season Listed", "Season Sold"
    ]]

to_drop = fayetteville_grouped_3_2[fayetteville_grouped_3_2["Property Type"] == "Duplex"].index
fayetteville_grouped_3_2 = fayetteville_grouped_3_2.drop(to_drop)
fayetteville_grouped_3_2


single_family_residence_3_2_fayetteville = fayetteville_grouped_3_2.loc[fayetteville_grouped_3_2['Property Type'] == 'Single Family Residence']
single_family_residence_3_2_fayetteville


In [ ]:
raleigh_grouped_3_2["Property Type"].value_counts()

In [ ]:
fayetteville_grouped_3_2["Property Type"].value_counts()

### Visualization of days on market per season listed

##### "DOM" = Days on market
##### "SZN" = Season

In [ ]:
def grouping_by_season(original_df, column_name):

    #creating new dataframe that is grouped by season listed and the average days on market for each season listed.
    df_to_order = original_df.groupby('Season Listed')[column_name].mean().reset_index()

    #creating list to use as template for displaying seasons in order (Spring-Summer-Fall-Winter)
    season_order = ['Spring', 'Summer', 'Fall', 'Winter']

    #converting season listed column to categorical type and then setting the order based on season_order list.
    df_to_order['Season Listed'] = pd.Categorical(df_to_order['Season Listed'], categories=season_order, ordered=True)

    #sorting the values now that they are categorized in the correct order
    df_to_order = df_to_order.sort_values('Season Listed')
    return(df_to_order)

In [ ]:
DOM_var = "Days On Market"
raleigh_DOM_SZN_grouped = grouping_by_season(original_df=raleigh_grouped_3_2, column_name=DOM_var)


plt.figure(figsize=(6, 8))
plt.xlabel("Season Listed")
plt.ylabel("Average days on market")
plt.bar(raleigh_DOM_SZN_grouped['Season Listed'], raleigh_DOM_SZN_grouped['Days On Market'])
plt.title("Raleigh NC Avg Days on market per season listed")
plt.show()


##### The chart above shows that the average days on market for Raleigh NC are highest in the fall, and lowest in the spring.

In [ ]:
fayetteville_DOM_SZN_grouped = grouping_by_season(original_df=combined_df, column_name=DOM_var)

plt.figure(figsize=(6, 8))
plt.xlabel("Season Listed")
plt.ylabel("Average days on market")
plt.bar(fayetteville_DOM_SZN_grouped['Season Listed'], fayetteville_DOM_SZN_grouped['Days On Market'])
plt.title("Fayetteville Avg Days on market per season listed")
plt.show()

##### The chart above shows a similar trend to that of Raleigh NC, with the highest average days on market for Fayetteville NC being in the fall, and the lowest in the spring.

### Visualization of Delta between List Price and sale price based on seasonality:

##### Creating new column called "Price Delta":

In [ ]:
raleigh_grouped_3_2["Price Delta"] = raleigh_grouped_3_2["Sold Price"] - raleigh_grouped_3_2["List Price"]
fayetteville_grouped_3_2["Price Delta"] = fayetteville_grouped_3_2["Sold Price"] - fayetteville_grouped_3_2["List Price"]
#raleigh_grouped_3_2

#### Price delta per season bar chart:

In [ ]:
delta_var = "Price Delta"
raleigh_delta_SZN_grouped = grouping_by_season(original_df=raleigh_grouped_3_2, column_name=delta_var)
plt.figure(figsize=(6, 8))
plt.xlabel("Season Listed")
plt.ylabel("Price Delta")
plt.bar(raleigh_delta_SZN_grouped['Season Listed'], raleigh_delta_SZN_grouped["Price Delta"])
plt.title("Raleigh NC Avg Price Delta per season listed")
plt.show()

##### The chart above shows that the highest price delta for Raleigh NC is in the spring, followed by winter and summer, with the fall showing the lowest price delta.

In [ ]:
fayetteville_DOM_SZN_grouped = grouping_by_season(original_df=fayetteville_grouped_3_2, column_name=delta_var)

plt.figure(figsize=(6, 8))
plt.xlabel("Season Listed")
plt.ylabel("Price Delta")
plt.bar(fayetteville_DOM_SZN_grouped['Season Listed'], fayetteville_DOM_SZN_grouped["Price Delta"])
plt.title("Fayetteville Avg Price Delta per season listed")
plt.show()

##### The chart above shows that similar to Raleigh, Fayetteville also has the highest price delta in the spring. However, the values for summer, fall, and winter are significantly lower compared to Raleigh.

# Analysis of Mortage rates and its effect on real estate sales 

#### Read the mortgage rate CSV file 

In [ ]:
# read all the mortage data from the CSV  File
mortageweekly_df = pd.read_excel("../Real-Estate-Analysis-Project/Data/weekly_mortgage_rates.xlsx",
                                 index_col="Week",
                                 parse_dates=True
                                 )
mortageweekly_df.head()

#### Clean up mortgage dataframe

In [ ]:
# Get the morrtgage data from 2021 ro 2014 
mortageweekly_df_sliced = mortageweekly_df.loc['2021-01-01':'2024-05-30']

#drop unwanted columns
mortageweekly_df_sliced.drop(columns=['30 years points','15 years FRM','15 years points','ARM','ARM points','margin','spread'],inplace=True)

mortageweekly_df_sliced.plot(figsize=[10,5])

In [ ]:
# add a column of week and year
mortageweekly_df_sliced["week-year"]  = mortageweekly_df_sliced.index.year.astype(str) + mortageweekly_df_sliced.index.isocalendar().week.astype(str)
mortageweekly_df_sliced.head()

#### Get single family homes real estate data 

In [ ]:
#single_family Raleigh
single_family_residence_3_2["week-year"]  = single_family_residence_3_2['List Date'].dt.year.astype(str) + single_family_residence_3_2['List Date'].dt.isocalendar().week.astype(str)
single_family_residence_3_2["Price Delta"] = single_family_residence_3_2["Sold Price"] - single_family_residence_3_2["List Price"]
single_family_residence_3_2.head() #162138 

#single_family Fayetteville
single_family_residence_3_2_fayetteville["week-year"]  = single_family_residence_3_2_fayetteville['List Date'].dt.year.astype(str) + single_family_residence_3_2_fayetteville['List Date'].dt.isocalendar().week.astype(str)
single_family_residence_3_2_fayetteville["Price Delta"] = single_family_residence_3_2_fayetteville["Sold Price"] - single_family_residence_3_2_fayetteville["List Price"]
single_family_residence_3_2_fayetteville.head() #162138 

#### Merge morgage data and single family real esate data frame on week-year of list date 

In [ ]:
#merge mortageweekly_df_sliced and single_family_residence_3_2 on week year column
realeastate_df  = pd.merge(mortageweekly_df_sliced,single_family_residence_3_2,on="week-year",how="inner")
realeastate_df_fayetteville  = pd.merge(mortageweekly_df_sliced,single_family_residence_3_2_fayetteville,on="week-year",how="inner")


In [ ]:
#Group by month and find average month data 
realeastate_monthly_df = realeastate_df.resample('M', on='List Date').agg({'Days On Market': ['mean'],"30 Years FRM":['mean'], "Price Delta":['mean']})

realeastate_monthly_df.columns = ['Avg Days on Market', 'Avg Intereset Rate','Avg Price Delta']

realeastate_monthly_df.head()

In [ ]:
#Group by month and find average month data 
realeastate_monthly_df_fayetteville = realeastate_df_fayetteville.resample('M', on='List Date').agg({'Days On Market': ['mean'],"30 Years FRM":['mean'], "Price Delta":['mean']})

realeastate_monthly_df_fayetteville.columns = ['Avg Days on Market', 'Avg Intereset Rate','Avg Price Delta']

realeastate_monthly_df_fayetteville.head()

### Visualize correlation between mortgage rate and the property days on market 

In [ ]:

#setting the fig size. 
fig, ax1 = plt.subplots(figsize=(12, 6))

# bar chart that displays average days on market for property in a month.
color = 'tab:blue'
ax1.set_xlabel('List Date')
ax1.set_ylabel('Days On Market', color=color)
ax1.plot(realeastate_monthly_df.index, realeastate_monthly_df['Avg Days on Market'], color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_title('Raleigh: Days On market and mortgage rate correlation')

# twin axes for the average mortage rate in a month
# telling the program that the axes will be plotted on the same graph. 
ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Avg Intereset Rate', color=color)
ax2.plot(realeastate_monthly_df.index, realeastate_monthly_df['Avg Intereset Rate'], color=color, marker='o')
ax2.tick_params(axis='y', labelcolor=color)


#tightening the layout
fig.tight_layout()  
plt.show()


In [ ]:

#setting the fig size. 
fig, ax1 = plt.subplots(figsize=(12, 6))

# bar chart that displays average days on market for property in a month.
color = 'tab:blue'
ax1.set_xlabel('List Date')
ax1.set_ylabel('Days On Market', color=color)
ax1.plot(realeastate_monthly_df_fayetteville.index, realeastate_monthly_df_fayetteville['Avg Days on Market'], color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_title('Fayetteville: Days On market and mortgage rate correlation')

# twin axes for the average mortage rate in a month
# telling the program that the axes will be plotted on the same graph. 
ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Avg Intereset Rate', color=color)
ax2.plot(realeastate_monthly_df_fayetteville.index, realeastate_monthly_df_fayetteville['Avg Intereset Rate'], color=color, marker='o')
ax2.tick_params(axis='y', labelcolor=color)


#tightening the layout
fig.tight_layout()  
plt.show()

##### Above graph indicate that the homes days on market has increased when the mortgage rate increased 

### Visualize correlation between mortgage rate and delta between list price and sales price 

In [ ]:


#setting the fig size. had to play around a bit to find a good fit.
fig, ax1 = plt.subplots(figsize=(12, 6))

# bar chart that displays average delta between list price and sold price 
color = 'tab:blue'
ax1.set_xlabel('List Date')
ax1.set_ylabel('Avg Price Delta', color=color)
ax1.plot(realeastate_monthly_df.index, realeastate_monthly_df['Avg Price Delta'], color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_title('Raleigh: List Price - Sold price delta and mortgage rate correlation')

# getting the array of values of y-axis
ticks = ax1.get_yticks()
# formatted the values into strings beginning with dollar sign
new_labels = [f'${amt:,.0f}' for amt in ticks] 
# Set the new labels
ax1.set_yticklabels(new_labels)

# twin axes for the average mortage rate in a month
# telling the program that the axes will be plotted on the same graph. 
ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Avg Intereset Rate', color=color)
ax2.plot(realeastate_monthly_df.index, realeastate_monthly_df['Avg Intereset Rate'], color=color, marker='o')
ax2.tick_params(axis='y', labelcolor=color)



#tightening the layout
fig.tight_layout()  
plt.show()



In [ ]:
#setting the fig size. had to play around a bit to find a good fit.
fig, ax1 = plt.subplots(figsize=(12, 6))

# bar chart that displays average delta between list price and sold price 
color = 'tab:blue'
ax1.set_xlabel('List Date')
ax1.set_ylabel('Avg Price Delta', color=color)
ax1.plot(realeastate_monthly_df_fayetteville.index, realeastate_monthly_df_fayetteville['Avg Price Delta'], color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_title('Fayetteville: List Price - Sold price delta and mortgage rate correlation')

# getting the array of values of y-axis
ticks = ax1.get_yticks()
# formatted the values into strings beginning with dollar sign
new_labels = [f'${amt:,.0f}' for amt in ticks] 
# Set the new labels
ax1.set_yticklabels(new_labels)
                    
# twin axes for the average mortage rate in a month
# telling the program that the axes will be plotted on the same graph. 
ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Avg Intereset Rate', color=color)
ax2.plot(realeastate_monthly_df_fayetteville.index, realeastate_monthly_df_fayetteville['Avg Intereset Rate'], color=color, marker='o')
ax2.tick_params(axis='y', labelcolor=color)


#tightening the layout
fig.tight_layout()  
plt.show()

##### There is no correlation between the mortgage rate and delta between list price and sold price 